# Notebook 2: Data Preprocessing & Feature Engineering

**Movie Recommendation System**  


Clean data, engineer features, and prepare train/test splits for modeling.

In [7]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os
import warnings
import re

warnings.filterwarnings('ignore')
pd.set_option('display.float_format', '{:.2f}'.format)

## 1. Load Data

In [8]:
# Setup paths
if os.path.basename(os.getcwd()) == 'notebooks':
    BASE_DIR = os.path.dirname(os.getcwd())
else:
    BASE_DIR = os.getcwd()

RAW_DIR = os.path.join(BASE_DIR, 'data', 'raw')
PROCESSED_DIR = os.path.join(BASE_DIR, 'data', 'processed')

os.makedirs(PROCESSED_DIR, exist_ok=True)

# Load datasets
ratings = pd.read_csv(os.path.join(RAW_DIR, 'ratings.csv'))
movies = pd.read_csv(os.path.join(RAW_DIR, 'movies.csv'))
tags = pd.read_csv(os.path.join(RAW_DIR, 'tags.csv'))

print(f"Loaded: {len(ratings):,} ratings, {len(movies):,} movies, {len(tags):,} tags")

Loaded: 25,000,095 ratings, 62,423 movies, 1,093,360 tags


## 2. Clean Tags for NLP

In [9]:
# Tag cleaning pipeline
print("Original tags:", len(tags))

tags_clean = tags.dropna(subset=['tag']).copy()
tags_clean['tag'] = tags_clean['tag'].str.lower().str.strip()
tags_clean = tags_clean[tags_clean['tag'] != '']
tags_clean = tags_clean[tags_clean['tag'].str.len() >= 2]
tags_clean['tag'] = tags_clean['tag'].apply(lambda x: re.sub(r'[^a-z0-9\s\-]', '', x))
tags_clean['tag'] = tags_clean['tag'].apply(lambda x: ' '.join(x.split()))
tags_clean = tags_clean.drop_duplicates(subset=['userId', 'movieId', 'tag'])

print(f"Cleaned tags: {len(tags_clean):,}")
print(f"Removed: {len(tags) - len(tags_clean):,} ({(len(tags) - len(tags_clean))/len(tags)*100:.1f}%)")
print(f"\nTop 10 tags:")
print(tags_clean['tag'].value_counts().head(10))

Original tags: 1093360
Cleaned tags: 1,092,394
Removed: 966 (0.1%)

Top 10 tags:
tag
sci-fi                8795
atmospheric           7053
action                6783
comedy                6370
surreal               5584
funny                 5366
based on a book       5196
twist ending          4904
visually appealing    4691
romance               4489
Name: count, dtype: int64


In [10]:
# Save cleaned tags
tags_clean.to_csv(os.path.join(PROCESSED_DIR, 'tags_clean.csv'), index=False)
print(f"Saved: {PROCESSED_DIR}/tags_clean.csv")

Saved: c:\Users\mhfou\Documents\MovieRecommenderSystem\data\processed/tags_clean.csv


## 3. Aggregate Tags per Movie

In [11]:
# Aggregate tags by movie
movie_tags = tags_clean.groupby('movieId')['tag'].apply(lambda x: ' '.join(x)).reset_index()
movie_tags.columns = ['movieId', 'tags_text']

# Add tag statistics
tag_counts = tags_clean.groupby('movieId').size().reset_index(name='tag_count')
unique_tags = tags_clean.groupby('movieId')['tag'].nunique().reset_index(name='unique_tag_count')

movie_tags = movie_tags.merge(tag_counts, on='movieId')
movie_tags = movie_tags.merge(unique_tags, on='movieId')

print(f"Movies with tags: {len(movie_tags):,}")
print(f"Avg tags per movie: {movie_tags['tag_count'].mean():.1f}")
print(f"Median: {movie_tags['tag_count'].median():.0f}")

Movies with tags: 45,249
Avg tags per movie: 24.1
Median: 5


In [12]:
# Save aggregated tags
movie_tags.to_csv(os.path.join(PROCESSED_DIR, 'movie_tags_aggregated.csv'), index=False)
print(f"Saved: {PROCESSED_DIR}/movie_tags_aggregated.csv")

Saved: c:\Users\mhfou\Documents\MovieRecommenderSystem\data\processed/movie_tags_aggregated.csv


## 4. Process Movies & Engineer Features

In [13]:
# Extract release year from title
def extract_year(title):
    match = re.search(r'\((\d{4})\)', title)
    return int(match.group(1)) if match else None

movies['release_year'] = movies['title'].apply(extract_year)

print(f"Movies with year: {movies['release_year'].notna().sum():,}")

Movies with year: 62,013


In [14]:
# Process genres
movies['genres_list'] = movies['genres'].str.split('|')
movies['genre_count'] = movies['genres_list'].apply(lambda x: len(x) if isinstance(x, list) else 0)
movies['has_genres'] = (movies['genres'] != '(no genres listed)').astype(int)

print(f"Movies with genres: {movies['has_genres'].sum():,}")

Movies with genres: 57,361


In [15]:
# Calculate rating statistics per movie
rating_stats = ratings.groupby('movieId').agg({
    'rating': ['count', 'mean', 'std', 'min', 'max']
}).reset_index()

rating_stats.columns = ['movieId', 'rating_count', 'rating_mean', 'rating_std', 'rating_min', 'rating_max']
rating_stats['rating_std'] = rating_stats['rating_std'].fillna(0)

print(f"Movies with ratings: {len(rating_stats):,}")

Movies with ratings: 59,047


In [16]:
# Calculate temporal features
ratings['datetime'] = pd.to_datetime(ratings['timestamp'], unit='s')

first_rating = ratings.groupby('movieId')['datetime'].min().reset_index()
first_rating.columns = ['movieId', 'first_rating_date']

last_rating = ratings.groupby('movieId')['datetime'].max().reset_index()
last_rating.columns = ['movieId', 'last_rating_date']

temporal_stats = first_rating.merge(last_rating, on='movieId')

# Calculate age and activity span
current_date = pd.Timestamp('2019-01-01')
temporal_stats['movie_age_days'] = (current_date - temporal_stats['first_rating_date']).dt.days
temporal_stats['activity_span_days'] = (temporal_stats['last_rating_date'] - temporal_stats['first_rating_date']).dt.days

print("Temporal features calculated")

Temporal features calculated


In [17]:
# Merge all features
movies_features = movies.copy()
movies_features = movies_features.merge(rating_stats, on='movieId', how='left')
movies_features = movies_features.merge(temporal_stats, on='movieId', how='left')
movies_features = movies_features.merge(movie_tags[['movieId', 'tag_count', 'unique_tag_count']], on='movieId', how='left')

# Fill missing values
movies_features['rating_count'] = movies_features['rating_count'].fillna(0)
movies_features['tag_count'] = movies_features['tag_count'].fillna(0)
movies_features['has_tags'] = (movies_features['tag_count'] > 0).astype(int)

# Calculate popularity score
movies_features['popularity_score'] = np.log1p(movies_features['rating_count'])

# Calculate rating velocity
movies_features['rating_velocity'] = movies_features['rating_count'] / (movies_features['activity_span_days'] + 1)
movies_features['rating_velocity'] = movies_features['rating_velocity'].fillna(0)

print(f"\nFinal features shape: {movies_features.shape}")
print(f"Total features: {movies_features.shape[1]} columns")


Final features shape: (62423, 21)
Total features: 21 columns


In [18]:
# Save processed movies
movies_features.to_csv(os.path.join(PROCESSED_DIR, 'movies_features.csv'), index=False)
print(f"Saved: {PROCESSED_DIR}/movies_features.csv")

Saved: c:\Users\mhfou\Documents\MovieRecommenderSystem\data\processed/movies_features.csv


## 5. Create Train/Test Split

In [19]:
# Temporal split: 80% earliest ratings for training, 20% most recent for testing
ratings_sorted = ratings.sort_values('timestamp')
split_idx = int(len(ratings_sorted) * 0.8)

train = ratings_sorted.iloc[:split_idx].copy()
test = ratings_sorted.iloc[split_idx:].copy()

print(f"Train: {len(train):,} ratings ({len(train)/len(ratings)*100:.1f}%)")
print(f"Test: {len(test):,} ratings ({len(test)/len(ratings)*100:.1f}%)")
print(f"\nTrain date range: {pd.to_datetime(train['timestamp'].min(), unit='s')} to {pd.to_datetime(train['timestamp'].max(), unit='s')}")
print(f"Test date range: {pd.to_datetime(test['timestamp'].min(), unit='s')} to {pd.to_datetime(test['timestamp'].max(), unit='s')}")

Train: 20,000,076 ratings (80.0%)
Test: 5,000,019 ratings (20.0%)

Train date range: 1995-01-09 11:46:49 to 2016-06-25 06:49:57
Test date range: 2016-06-25 06:50:00 to 2019-11-21 09:15:03


In [20]:
# Analyze cold-start problem
train_users = set(train['userId'].unique())
test_users = set(test['userId'].unique())
train_movies = set(train['movieId'].unique())
test_movies = set(test['movieId'].unique())

cold_start_users = test_users - train_users
cold_start_movies = test_movies - train_movies

print("Cold-start analysis:")
print(f"Users only in test: {len(cold_start_users):,} ({len(cold_start_users)/len(test_users)*100:.1f}%)")
print(f"Movies only in test: {len(cold_start_movies):,} ({len(cold_start_movies)/len(test_movies)*100:.1f}%)")

Cold-start analysis:
Users only in test: 24,658 (79.4%)
Movies only in test: 24,586 (44.5%)


In [21]:
# Calculate sparsity
train_sparsity = (1 - len(train) / (train['userId'].nunique() * train['movieId'].nunique())) * 100
test_sparsity = (1 - len(test) / (test['userId'].nunique() * test['movieId'].nunique())) * 100

print(f"\nTrain sparsity: {train_sparsity:.2f}%")
print(f"Test sparsity: {test_sparsity:.2f}%")


Train sparsity: 99.58%
Test sparsity: 99.71%


In [22]:
# Save train and test sets
train[['userId', 'movieId', 'rating', 'timestamp']].to_csv(os.path.join(PROCESSED_DIR, 'train.csv'), index=False)
test[['userId', 'movieId', 'rating', 'timestamp']].to_csv(os.path.join(PROCESSED_DIR, 'test.csv'), index=False)

print(f"\nSaved: {PROCESSED_DIR}/train.csv")
print(f"Saved: {PROCESSED_DIR}/test.csv")


Saved: c:\Users\mhfou\Documents\MovieRecommenderSystem\data\processed/train.csv
Saved: c:\Users\mhfou\Documents\MovieRecommenderSystem\data\processed/test.csv


## 6. Summary

In [23]:
# Print summary
print("PREPROCESSING COMPLETE")
print("\nProcessed files:")
print(f"- tags_clean.csv: {len(tags_clean):,} cleaned tags")
print(f"- movie_tags_aggregated.csv: {len(movie_tags):,} movies with aggregated tags")
print(f"- movies_features.csv: {len(movies_features):,} movies with {movies_features.shape[1]} features")
print(f"- train_ratings.csv: {len(train):,} ratings (80%)")
print(f"- test_ratings.csv: {len(test):,} ratings (20%)")

print("\nFeature coverage:")
print(f"- Movies with genres: {movies_features['has_genres'].sum():,} ({movies_features['has_genres'].sum()/len(movies_features)*100:.1f}%)")
print(f"- Movies with tags: {movies_features['has_tags'].sum():,} ({movies_features['has_tags'].sum()/len(movies_features)*100:.1f}%)")
print(f"- Movies with ratings: {(movies_features['rating_count'] > 0).sum():,} ({(movies_features['rating_count'] > 0).sum()/len(movies_features)*100:.1f}%)")

print("\nCold-start scenarios:")
print(f"- Users in test only: {len(cold_start_users):,}")
print(f"- Movies in test only: {len(cold_start_movies):,}")

print("\nReady for:")
print("- Content-based filtering (genres + NLP)")
print("- Collaborative filtering")
print("- Matrix factorization (SVD)")
print("- Hybrid model")

PREPROCESSING COMPLETE

Processed files:
- tags_clean.csv: 1,092,394 cleaned tags
- movie_tags_aggregated.csv: 45,249 movies with aggregated tags
- movies_features.csv: 62,423 movies with 21 features
- train_ratings.csv: 20,000,076 ratings (80%)
- test_ratings.csv: 5,000,019 ratings (20%)

Feature coverage:
- Movies with genres: 57,361 (91.9%)
- Movies with tags: 45,249 (72.5%)
- Movies with ratings: 59,047 (94.6%)

Cold-start scenarios:
- Users in test only: 24,658
- Movies in test only: 24,586

Ready for:
- Content-based filtering (genres + NLP)
- Collaborative filtering
- Matrix factorization (SVD)
- Hybrid model
